# CyberLeaks DW (MVP)

_Autor:_ André Serrano

## 1. Descrição

O CyberLeaks DW é uma solução de Data Warehouse desenvolvida para analisar, monitorar e detectar padrões em vazamentos de dados. Utilizando um modelo estrela, a plataforma estrutura informações sobre violações de segurança, permitindo consultas avançadas e insights estratégicos para mitigar riscos cibernéticos.

## 2. Objetivo

O CyberLeaks DW resolve esses desafios ao estruturar os dados sobre vazamentos em um Data Warehouse, facilitando a análise por meio de consultas SQL otimizadas e integração com ferramentas de visualização de dados. A solução inclui:
- Coleta e armazenamento de dados do Kaggle.
- Modelo estrela para organizar as informações e permitir consultas eficientes.
- Pipeline ETL para extração, transformação e carga dos dados.
- Análises e insights sobre frequência, setores afetados e métodos de ataque.

## 3. Implementação

A implementação visa criar uma estrutura robusta para análise de vazamentos de dados, utilizando um Data Warehouse baseado no modelo estrela. Esse processo permitirá consultas otimizadas, análises preditivas e identificação de padrões de ataques cibernéticos.

A seguir mostraremos a implementação e no final apresentaremos respostas de algumas perguntas interessantes para temática.

### 3.1. Base de dados (Kaggle)

A base de dados utilizada foi obtida no Kaggle, a partir do dataset "Data Breaches - A Comprehensive List", disponível em:

[Kaggle: Data Breaches - A Comprehensive List](https://www.kaggle.com/datasets/thedevastator/data-breaches-a-comprehensive-list)

Esse conjunto de dados fornece um histórico detalhado sobre vazamentos de dados, abrangendo diversas organizações e setores, permitindo análises sobre a frequência, magnitude e causas desses incidentes ao longo dos anos.

O dataset contém informações estruturadas sobre vazamentos de dados ocorridos desde o ano 2004, abrangendo os seguintes atributos:

| **Atributo**            | **Descrição** |
|-------------------------|--------------|
| `Entity`               | Nome da organização que sofreu o vazamento de dados. |
| `Year`                 | Ano em que o vazamento ocorreu. |
| `Records`              | Número de registros comprometidos. |
| `Organization type`    | Setor da organização (ex: Tecnologia, Saúde, Governo, Educação). |
| `Method`               | Método pelo qual os dados foram vazados (ex: hacking, erro humano, malware). |
| `Sources`              | Fonte da informação sobre o vazamento (artigos, relatórios, investigações públicas). |

In [0]:
# Instalando os pacote kaggle para baixar o dataset pela API
%pip install kaggle -q

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import json
import os

# Função de autenticação na plataforma Kaggle
def kaggle_auth_setup(kaggle_auth):
    try:
        # Cria o diretório "~/.kaggle"
        os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
        output_file = os.path.expanduser("~/.kaggle/kaggle.json")

        # Cria o arquivo "kaggle.json" com as credenciais
        with open(output_file, "w") as file:
            json.dump(json.loads(kaggle_auth), file, indent=4)
        
        os.chmod(output_file, 0o600)  # Define permissões seguras
        print("Autenticação Kaggle configurada com sucesso!")
    
    except Exception as e:
        print(f"Erro na autenticação: {e}")

In [0]:
# Realiza a autenticação com as credenciais informadas (Token API)
kaggle_auth_setup('{"username":"andreserrano","key":"XXXXXXX"}')

Autenticação Kaggle configurada com sucesso!


In [0]:
# Visualizando o json criado
!cat ~/.kaggle/kaggle.json

{
    "username": "andreserrano",
    "key": "XXXXXXX"
}

In [0]:
# Realizando o download do dataset data-breaches-a-comprehensive-list
!kaggle datasets download -d thedevastator/data-breaches-a-comprehensive-list

Dataset URL: https://www.kaggle.com/datasets/thedevastator/data-breaches-a-comprehensive-list
License(s): CC0-1.0


In [0]:
# Listando os arquivos para verificar se o download foi realizado com sucesso (data-breaches-a-comprehensive-list.zip)
dbutils.fs.ls('file:/databricks/driver')

Out[5]: [FileInfo(path='file:/databricks/driver/preload_class.lst', name='preload_class.lst', size=1306936, modificationTime=1742827285593),
 FileInfo(path='file:/databricks/driver/azure/', name='azure/', size=4096, modificationTime=1742827285575),
 FileInfo(path='file:/databricks/driver/hadoop_accessed_config.lst', name='hadoop_accessed_config.lst', size=2755, modificationTime=1742827285580),
 FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096, modificationTime=1742827284779),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096, modificationTime=1742838345495),
 FileInfo(path='file:/databricks/driver/data-breaches-a-comprehensive-list.zip', name='data-breaches-a-comprehensive-list.zip', size=9067, modificationTime=1668096970000),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096, modificationTime=1742838360620)]

In [0]:
# Descompactando o arquivo data-breaches-a-comprehensive-list.zip
!unzip /databricks/driver/data-breaches-a-comprehensive-list.zip

Archive:  /databricks/driver/data-breaches-a-comprehensive-list.zip
  inflating: df_1.csv                


In [0]:

# Listando os arquivos
dbutils.fs.ls('file:/databricks/driver')

Out[7]: [FileInfo(path='file:/databricks/driver/preload_class.lst', name='preload_class.lst', size=1306936, modificationTime=1742827285593),
 FileInfo(path='file:/databricks/driver/azure/', name='azure/', size=4096, modificationTime=1742827285575),
 FileInfo(path='file:/databricks/driver/hadoop_accessed_config.lst', name='hadoop_accessed_config.lst', size=2755, modificationTime=1742827285580),
 FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096, modificationTime=1742827284779),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096, modificationTime=1742838345495),
 FileInfo(path='file:/databricks/driver/df_1.csv', name='df_1.csv', size=23269, modificationTime=1668096968000),
 FileInfo(path='file:/databricks/driver/data-breaches-a-comprehensive-list.zip', name='data-breaches-a-comprehensive-list.zip', size=9067, modificationTime=1668096970000),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096, modificationTime=174283

In [0]:
# Renomea e move o dataset df_1.csv para data-breaches-a-comprehensive-list.csv
dbutils.fs.mv('file:/databricks/driver/df_1.csv', 'dbfs:/Filestore/data-breaches-a-comprehensive-list.csv', recurse=True)

Out[8]: True

In [0]:
# Lista o arquivo para verificar se foi renomeado o CSV
dbutils.fs.ls('dbfs:/Filestore')

Out[9]: [FileInfo(path='dbfs:/Filestore/data-breaches-a-comprehensive-list.csv', name='data-breaches-a-comprehensive-list.csv', size=23269, modificationTime=1742838500000)]


### 3.2. Coleta

A coleta de dados foi realizada a partir do dataset "Data Breaches - A Comprehensive List", disponível no Kaggle. Esse conjunto de dados foi baixado e armazenado na nuvem, garantindo acessibilidade para processamento e análise. Durante essa etapa, os dados passaram por um processo de verificação para assegurar sua integridade, incluindo a remoção de duplicatas, a conversão de tipos de dados e o tratamento de valores ausentes. Caso necessário, futuras expansões poderão incluir a extração de novas fontes de dados, como relatórios de segurança pública e bases de dados governamentais sobre incidentes de vazamento.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Criar sessão Spark
spark = SparkSession.builder.appName("CyberLeaks_Coleta").getOrCreate()

# oletar os dados do arquivo CSV
df_spark = spark.read.csv(
    'dbfs:/Filestore/data-breaches-a-comprehensive-list.csv', 
    header=True, 
    inferSchema=True  # Permite inferir automaticamente os tipos de dados
)

# Renomear colunas para padronização e evitar espaços
df_spark = df_spark.withColumnRenamed("Organization type", "Organization_Type") \
                   .withColumnRenamed("Method", "Method_Name") \
                   .withColumnRenamed("Sources", "Source_Details")

# Remover registros duplicados
df_spark = df_spark.dropDuplicates()

# Remover registros com valores ausentes
df_spark = df_spark.dropna()

# Converter tipos de dados
df_spark = df_spark.withColumn("Year", col("Year").cast("int"))
df_spark = df_spark.withColumn("Records", col("Records").cast("bigint"))

# Verificar estrutura dos dados
df_spark.printSchema()  # Exibe os tipos de dados inferidos

# Exibir uma amostra dos dados coletados
df_spark.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- Entity: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Records: long (nullable = true)
 |-- Organization_Type: string (nullable = true)
 |-- Method_Name: string (nullable = true)
 |-- Source_Details: string (nullable = true)

+---+--------------------+----+---------+-----------------+-------------------+--------------+
|_c0|              Entity|Year|  Records|Organization_Type|        Method_Name|Source_Details|
+---+--------------------+----+---------+-----------------+-------------------+--------------+
|  5|Advocate Medical ...|2017|  4000000|       healthcare|lost / stolen media|      [13][14]|
|  4|          Adobe Inc.|2019|  7500000|             tech|      poor security|      [11][12]|
|  3|Adobe Systems Inc...|2013|152000000|             tech|             hacked|          [10]|
|  1|               500px|2020| 14870304|social networking|             hacked|           [7]|
|  0|21st Century Onco...|2016|  2200000|  

### 3.3. Modelagem

A etapa de Modelagem foi estruturada utilizando um **modelo estrela**, garantindo um armazenamento eficiente e otimizado para consultas analíticas. A modelagem foi definida com uma **tabela fato** (`fatos_ocorrencias`), que centraliza os registros de vazamentos de dados, e **quatro tabelas dimensão** (`dim_entidade`, `dim_organizacao`, `dim_metodo`, `dim_fonte`), que armazenam informações categóricas essenciais para análise. Essa abordagem facilita a realização de consultas rápidas e permite explorar as relações entre **empresas afetadas, setores vulneráveis, métodos de ataque e fontes de informação**. A definição dos atributos e relacionamentos foi documentada no **Dicionário de Dados**, garantindo padronização e transparência na estrutura do Data Warehouse.


![Diagrama DW](files/shared_uploads/guibispo@gmail.com/DW_Violacoes_Seguranca.png)


#### Tabela Fato: `fatos_ocorrencias`
| **Atributo**       | **Tipo**    | **Descrição** |
|--------------------|------------|--------------|
| `Entity_ID`       | INT        | Identificador da entidade afetada pelo vazamento (FK). |
| `Year`            | INT        | Ano em que ocorreu o vazamento. |
| `Records`         | BIGINT     | Número de registros comprometidos. |
| `Method_ID`       | INT        | Identificador do método utilizado no vazamento (FK). |
| `Organization_ID` | INT        | Identificador do setor da organização afetada (FK). |
| `Source_ID`       | INT        | Identificador da fonte da informação sobre o vazamento (FK). |

---

#### Tabela Dimensão: `dim_entidade`
| **Atributo**    | **Tipo**  | **Descrição** |
|---------------|----------|--------------|
| `Entity_ID`   | INT      | Identificador único da entidade (PK). |
| `Entity_Name` | STRING   | Nome da organização afetada pelo vazamento. |

---

#### Tabela Dimensão: `dim_organizacao`
| **Atributo**        | **Tipo**  | **Descrição** |
|--------------------|----------|--------------|
| `Organization_ID`  | INT      | Identificador único do setor (PK). |
| `Organization_Type` | STRING   | Tipo de organização (ex: Governo, Saúde, Tecnologia). |

---

#### Tabela Dimensão: `dim_metodo`
| **Atributo**    | **Tipo**  | **Descrição** |
|---------------|----------|--------------|
| `Method_ID`   | INT      | Identificador único do método de ataque (PK). |
| `Method_Name` | STRING   | Nome do método de ataque (ex: Hacking, Erro Humano). |

---

#### Tabela Dimensão: `dim_fonte`
| **Atributo**       | **Tipo**  | **Descrição** |
|-------------------|----------|--------------|
| `Source_ID`      | INT      | Identificador único da fonte (PK). |
| `Source_Details` | STRING   | Descrição da fonte da informação sobre o vazamento. |



### 3.4. ETL (Extração, Transformação e Carga)

A etapa de ETL (Extração, Transformação e Carga) tem como objetivo estruturar os dados coletados e prepará-los para análises eficientes dentro do Data Warehouse. Inicialmente, os dados passam por uma fase de transformação, onde são aplicadas conversões de tipos para garantir consistência e integridade. Em seguida, as tabelas dimensão são criadas, removendo duplicatas e gerando identificadores únicos para cada entidade, organização, método de ataque e fonte de informação. A tabela fato é então construída, associando os registros de vazamentos de dados com as tabelas dimensão, permitindo a realização de consultas otimizadas. Por fim, as tabelas processadas são armazenadas no formato Parquet, garantindo um acesso rápido e eficiente dentro do ambiente Databricks.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Criar sessão Spark para o processo ETL
spark = SparkSession.builder.appName("CyberLeaks_ETL").getOrCreate()

# Corrigir tipos de dados
df_spark = df_spark.withColumn("Year", col("Year").cast("int"))
df_spark = df_spark.withColumn("Records", col("Records").cast("bigint"))

# Criar tabelas dimensão (removendo duplicatas e gerando IDs únicos)
dim_entidade = df_spark.select(col("Entity").alias("Entity_Name")).distinct() \
    .withColumn("Entity_ID", monotonically_increasing_id())

dim_organizacao = df_spark.select("Organization_Type").distinct() \
    .withColumn("Organization_ID", monotonically_increasing_id())

dim_metodo = df_spark.select("Method_Name").distinct() \
    .withColumn("Method_ID", monotonically_increasing_id())

dim_fonte = df_spark.select("Source_Details").distinct() \
    .withColumn("Source_ID", monotonically_increasing_id())

# Criar a tabela fato associando IDs das dimensões
fatos_ocorrencias = df_spark \
    .join(dim_entidade, df_spark["Entity"] == dim_entidade["Entity_Name"], "left") \
    .join(dim_organizacao, df_spark["Organization_Type"] == dim_organizacao["Organization_Type"], "left") \
    .join(dim_metodo, df_spark["Method_Name"] == dim_metodo["Method_Name"], "left") \
    .join(dim_fonte, df_spark["Source_Details"] == dim_fonte["Source_Details"], "left") \
    .select(
        col("Entity_ID"), 
        col("Year"), 
        col("Records"), 
        col("Method_ID"), 
        col("Organization_ID"), 
        col("Source_ID")
    )

# Criar tabelas temporárias no Spark SQL para análises futuras
dim_entidade.createOrReplaceTempView("dim_entidade")
dim_organizacao.createOrReplaceTempView("dim_organizacao")
dim_metodo.createOrReplaceTempView("dim_metodo")
dim_fonte.createOrReplaceTempView("dim_fonte")
fatos_ocorrencias.createOrReplaceTempView("fatos_ocorrencias")

# Salvar tabelas como Parquet para otimização no Databricks
dim_entidade.write.mode("overwrite").parquet("dbfs:/mnt/data/dim_entidade")
dim_organizacao.write.mode("overwrite").parquet("dbfs:/mnt/data/dim_organizacao")
dim_metodo.write.mode("overwrite").parquet("dbfs:/mnt/data/dim_metodo")
dim_fonte.write.mode("overwrite").parquet("dbfs:/mnt/data/dim_fonte")
fatos_ocorrencias.write.mode("overwrite").parquet("dbfs:/mnt/data/fatos_ocorrencias")

print("ETL finalizado com sucesso!")

ETL finalizado com sucesso!



#### 3.5. Análise

A etapa de **Análise** consiste na execução de **consultas SQL** para extrair insights estratégicos sobre vazamentos de dados. As consultas são formuladas para responder a perguntas-chave, como **quais setores são mais vulneráveis**, **quais métodos de ataque são mais frequentes**, **quais empresas tiveram os maiores vazamentos** e **como esses incidentes evoluíram ao longo do tempo**. Utilizando a estrutura do **modelo estrela**, é possível explorar os dados de maneira eficiente, combinando a tabela fato (`fatos_ocorrencias`) com as tabelas dimensão (`dim_entidade`, `dim_organizacao`, `dim_metodo` e `dim_fonte`). Os resultados dessas análises permitem identificar padrões, avaliar riscos e contribuir para estratégias de mitigação de vazamentos de dados.


In [0]:
%sql
-- 1. Qual foi o ano com o maior número de registros vazados?
SELECT Year, SUM(Records) AS total_records
FROM fatos_ocorrencias
GROUP BY Year
ORDER BY total_records DESC
LIMIT 1;

Year,total_records
2019,3724900831


In [0]:
%sql
-- 2. Qual a tendência de vazamentos ao longo dos anos?
SELECT Year, SUM(Records) AS total_records
FROM fatos_ocorrencias
GROUP BY Year
ORDER BY Year ASC;


Year,total_records
null,2363000
2004,92510000
2005,46825000
2006,71260000
2007,153286405
2008,69066500
2009,255467987
2010,15980476
2011,227788137
2012,428839635


In [0]:
%sql
-- 3. Quais são as 5 empresas mais afetadas por vazamentos de dados?
SELECT e.Entity_Name, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_entidade e ON f.Entity_ID = e.Entity_ID
GROUP BY e.Entity_Name
ORDER BY total_records DESC
LIMIT 5;


Entity_Name,total_records
Yahoo,3500000000
First American Corporation,885000000
Facebook,864500000
Marriott International,505200000
MongoDB,477000000


In [0]:
%sql
-- 4. Quais setores tiveram mais vazamentos de dados?
SELECT o.Organization_Type, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_organizacao o ON f.Organization_ID = o.Organization_ID
GROUP BY o.Organization_Type
ORDER BY total_records DESC;


Organization_Type,total_records
web,5203695724
social network,1238000000
tech,1000898000
financial service company,885000000
financial,818597083
hotel,505563000
retail,372140740
data broker,340000000
telecommunications,320000000
Telephone directory,299055000


In [0]:
%sql
-- 5. Qual setor foi mais afetado a cada ano?
SELECT Year, o.Organization_Type, SUM(Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_organizacao o ON f.Organization_ID = o.Organization_ID
GROUP BY Year, o.Organization_Type
ORDER BY Year, total_records DESC;


Year,Organization_Type,total_records
null,restaurant,2000000
null,hotel,363000
null,transport,null
2004,web,92000000
2004,shopping,510000
2005,financial,45425000
2005,retail,1400000
2006,"government, military",26500000
2006,telecoms,21000000
2006,web,20000000


In [0]:
%sql
-- 6. Quais são os métodos de ataque mais comuns?
SELECT m.Method_Name, COUNT(*) AS ocorrencias, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_metodo m ON f.Method_ID = m.Method_ID
GROUP BY m.Method_Name
ORDER BY total_records DESC;


Method_Name,ocorrencias,total_records
hacked,190,7349780062
poor security,43,3610143028
unknown,7,448233889
poor security / hacked,1,412214295
accidentally published,20,269917485
data exposed by misconfiguration,1,250000000
Poor security,1,201000000
lost / stolen media,33,170434536
unsecured S3 bucket,1,106000000
unprotected api,1,100000000


In [0]:
%sql
-- 7. Como os métodos de ataque evoluíram ao longo do tempo?
SELECT Year, m.Method_Name, COUNT(*) AS ocorrencias, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_metodo m ON f.Method_ID = m.Method_ID
GROUP BY Year, m.Method_Name
ORDER BY Year ASC, total_records DESC;


Year,Method_Name,ocorrencias,total_records
null,hacked,3,2363000
2004,"inside job, hacked",1,92000000
2004,inside job,1,510000
2005,hacked,2,41400000
2005,lost / stolen media,3,5300000
2005,poor security,1,125000
2006,lost / stolen computer,1,26500000
2006,accidentally published,1,20000000
2006,lost / stolen media,2,17200000
2006,hacked,1,4000000


In [0]:
%sql
-- 8. Quais métodos de ataque afetam mais cada setor?
SELECT o.Organization_Type, m.Method_Name, COUNT(*) AS ocorrencias, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_organizacao o ON f.Organization_ID = o.Organization_ID
JOIN dim_metodo m ON f.Method_ID = m.Method_ID
GROUP BY o.Organization_Type, m.Method_Name
ORDER BY o.Organization_Type, total_records DESC;


Organization_Type,Method_Name,ocorrencias,total_records
Clinical Laboratory,poor security,1,11900000
Consumer Goods,hacked,1,150000000
Information Security,hacked,1,null
Network Monitoring,hacked,1,null
QR code payment,"improper setting, hacked",1,20076016
Question & Answer,hacked,1,100000000
Telephone directory,unknown,1,299055000
academic,hacked,9,2725540
academic,lost / stolen media,1,2200000
academic,lost / stolen computer,2,2172000


In [0]:
%sql
-- 9. Quais são as principais fontes de vazamento de dados?
SELECT s.Source_Details, COUNT(*) AS ocorrencias, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_fonte s ON f.Source_ID = s.Source_ID
GROUP BY s.Source_Details
ORDER BY total_records DESC;


Source_Details,ocorrencias,total_records
[395][396],1,3000000000
[153],1,885000000
[146][147],1,540000000
[234],1,500000000
[397][398][399][400][401],1,500000000
[157][158],1,412214295
[134],1,340000000
[18],1,320000000
[341][342],1,299055000
[248],1,275000000


In [0]:
%sql
-- 10. Fontes de vazamento mais relevantes por setor
SELECT o.Organization_Type, s.Source_Details, COUNT(*) AS ocorrencias, SUM(f.Records) AS total_records
FROM fatos_ocorrencias f
JOIN dim_organizacao o ON f.Organization_ID = o.Organization_ID
JOIN dim_fonte s ON f.Source_ID = s.Source_ID
GROUP BY o.Organization_Type, s.Source_Details
ORDER BY o.Organization_Type, total_records DESC;


Organization_Type,Source_Details,ocorrencias,total_records
Clinical Laboratory,[279],1,11900000
Consumer Goods,[358],1,150000000
Information Security,[154][155][156],1,null
Network Monitoring,[301],1,null
QR code payment,[276],1,20076016
Question & Answer,[280],1,100000000
Telephone directory,[341][342],1,299055000
academic,[36],5,5176000
academic,[357],1,1164540
academic,[361],1,300000


In [0]:
%sql
-- 11. Quais métodos de ataque estão crescendo mais rápido?
WITH metodo_crescimento AS (
    SELECT Year, m.Method_Name, SUM(f.Records) AS total_records,
           LAG(SUM(f.Records)) OVER (PARTITION BY m.Method_Name ORDER BY Year) AS prev_year_records
    FROM fatos_ocorrencias f
    JOIN dim_metodo m ON f.Method_ID = m.Method_ID
    GROUP BY Year, m.Method_Name
)
SELECT Year, Method_Name, total_records, (total_records - prev_year_records) AS crescimento
FROM metodo_crescimento
WHERE prev_year_records IS NOT NULL
ORDER BY crescimento DESC;


Year,Method_Name,total_records,crescimento
2013,hacked,3460570298,3056129455
2019,poor security,2617208000,2166618000
2018,hacked,986975943,901228066
2018,poor security,450590000,285571000
2019,unknown,299055000,206771111
2012,hacked,404440843,205705005
2011,hacked,198735838,192915438
2017,poor security,165019000,164989000
2009,hacked,176521778,149021778
2007,hacked,106100000,102100000
